# 03 SDK | 03 JSON LLM Response 

## JSON mode

Azure OpenAI JSON mode is a feature that instructs models from the Azure OpenAI GPT family to return results in JSON format, providing a structured and easily parsable output. This allows developers to seamlessly integrate OpenAI's language models into their applications, facilitating efficient handling and utilization of the generated content to exchange data with other systems or call native functions.


### Further information

[What is JSON mode](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/json-mode)

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))


## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.


In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI1106_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI1106_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI1106_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";

string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Client created...");

Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## Step 2: Compose ChatCompletionsOptions

Each chat would follow similar structure, where _System_, _Agent_ and _User_ messages are added in sequence. Parameters, such as _Temperature_ could be set per call.
The model would then response with the most likely next message. There are two key factors that need to be present to successfully use JSON mode:
- `ResponseFormat = ChatCompletionsResponseFormat.JsonObject`
- **and** the model is instructed within the `systemMessage` to output JSON 

In [ ]:
//Define System Prompt
string systemMessage = @" 
    You are an assistant who helps users to identify the most important keywords 
    from unstructured text. 
    You return results in valid JSON format. 
    You return the recognized keywords as an array with the key 'keywords'
"; 

string userMessage = @"
    I need information regarding my order 4711 from last month. My customer number is AE-100-10?
";

//Compose Chat (Simplified - No few shot learning in this example)
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions()
{
    ResponseFormat = ChatCompletionsResponseFormat.JsonObject,
    DeploymentName = chatCompletionDeploymentName,
    Messages =
    {
        new ChatRequestSystemMessage(systemMessage),
        new ChatRequestUserMessage(userMessage),
    },
    MaxTokens = 100, 
    NucleusSamplingFactor = 0.1f,
    Temperature = 0.1f, 
    ChoiceCount = 1
};


Console.WriteLine("ChatCompletionsOptions created...");

Expected output:

```
ChatCompletionsOptions created...
```

## Step 3: Call ChatCompletion API

The API will response in chunks of tokens. One can display them as outlined in this example.

In [ ]:
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);

Console.WriteLine($"Response: \n {response.Value.Choices[0].Message.Content}");

Expected output:

```
Response: 
 
{
    "keywords": ["order 4711", "last month", "customer number", "AE-100-10"]
}
```

## Next Steps

- The concept of Embeddings allows transforming information into a numerical representation preserving the semantic context of the information: [Demo Embeddings](../04_Embeddings/01_BasicEmbeddings.ipynb)